In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20182640\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Pre-Processing Texts Part 2

We start with importing the necessary libraries and the the module

In [112]:
from preprocessor import Preprocessor
import preprocessor

import os
import pandas as pd

ROOT_DIR = preprocessor.ROOT_DIR
DATA_PATH = preprocessor.DATA_PATH

preprocessor = Preprocessor(ROOT_DIR)

c:\Users\20182640\.virtualenvs\Text-Mining-xR8YyNgY\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# 1 Importing and cleaning

## 1.1. Importing the raw text data from the XLSX file

In [113]:
df_events_summary = pd.read_excel(os.path.join(DATA_PATH, "component_2", "initiating-events-summary-2021.xlsx"), sheet_name="2021 IE Events")

df_events_summary.head()

,Docket,LERNuma,EventDt,IPFCode,IPFShortDesc,A_Blackout,B_LOOP,C_Bus,D_Pneumatic,E_CoolingWater,...,L3_BypUnavailable,MX_ATWS,NX_ISLOCA,PX_LossOfFeed,RptName,PLANT_TYPE,VENDOR,UnitMode,PowerLevel,Abstract
0,29,291987003.0,1987-02-18,QR3,Reactivity Control Imbalance,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yankee-Rowe,P,WE,PO,100.0,"On 2/18/87, at 0001 hours, during normal stead..."
1,29,291987012.0,1987-07-06,QR0,RCS High Pressure (RPS Trip),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yankee-Rowe,P,WE,SU,1.0,"On July 6, 1987, with the plant in Mode 2 at 5..."
2,29,291988002.0,1988-03-22,QR8,Spurious Reactor Trip,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yankee-Rowe,P,WE,PO,100.0,"On 3/22/88, at 0042, with the Plant operating ..."
3,29,291988003.0,1988-03-26,QP2,Partial Loss of Feedwater Flow,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yankee-Rowe,P,WE,PO,85.0,"On March 26, 1988, while at 85% power and retu..."
4,29,291988008.0,1988-05-17,QR5,Turbine Trip,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yankee-Rowe,P,WE,PO,100.0,"At 2323 hours, 17 May 1988, in Mode 1 at full ..."


## 1.2. Cleaning the raw data
We will drop the not necessary columns from the dataframe, rename the columsn we need and clean the texts from the description column


In [114]:
df_events_short = df_events_summary[["Abstract"]]

# Some checks and print statements
print("There are duplicates in the data") if any(df_events_short.duplicated()) else print("There are no duplicates in the data")
print(f"There are {len(df_events_short)} unique events in the data")

# Remove duplicates from data
df_events_short = df_events_short.drop_duplicates()
df_events_short = df_events_short.dropna()
df_events_short = df_events_short.reset_index(drop=True)
print(f"There are {len(df_events_short)} unique events in the data after dropping duplicates and nan")

# Rename the columns
df_events_short.columns = ["description"]

df_events_short.head()

There are duplicates in the data
There are 3791 unique events in the data
There are 3713 unique events in the data after dropping duplicates and nan


,description
0,"On 2/18/87, at 0001 hours, during normal stead..."
1,"On July 6, 1987, with the plant in Mode 2 at 5..."
2,"On 3/22/88, at 0042, with the Plant operating ..."
3,"On March 26, 1988, while at 85% power and retu..."
4,"At 2323 hours, 17 May 1988, in Mode 1 at full ..."


In [115]:
# For all description in every row, clean the text
print(f"There are {len(df_events_short)} unique events in the data before cleaning html")
df_events_short_clean = preprocessor.clean_html(df_events_short, "description")
print(f"There are {len(df_events_short_clean)} unique events in the data after cleaning html")
df_events_short_clean.head()

There are 3713 unique events in the data before cleaning html
Cleaning data with ftfy...


100%|██████████| 3713/3713 [00:01<00:00, 2592.14it/s]

There are 3713 unique events in the data after cleaning html


,description
0,"On 2/18/87, at 0001 hours, during normal stead..."
1,"On July 6, 1987, with the plant in Mode 2 at 5..."
2,"On 3/22/88, at 0042, with the Plant operating ..."
3,"On March 26, 1988, while at 85% power and retu..."
4,"At 2323 hours, 17 May 1988, in Mode 1 at full ..."


## 1.3 Save the data to a text file
The text file can be imported as one file to label studio which makes  it easier because of the cap of 100 files per import

In [116]:
# Save the column as a text file
import numpy as np


np.savetxt(os.path.join(DATA_PATH, "component_2", "initiating-events-summary-2021.txt"), df_events_short["description"].values, fmt='%s', encoding='utf-8')